# Clase 21

Para una mejor visualización entrar al siguiente [link](https://nbviewer.jupyter.org/github/racsosabe/Miscelanea/blob/master/UPC/Clase%2021%20-%20Grafos%20IX.ipynb)

**Nota:** La clase 20 fue repasos de problemas diversos.

# Requisitos Previos

* Matemática Básica
* Matemática Discreta
* Grafos

# All-Pairs Shortest Paths

En las clases anteriores vimos cómo hallar el camino más corto desde un nodo origen a los demás en un grafo dirigido y ponderado $G = (V, E)$; esta vez, analizaremos cómo determinar el camino más corto entre cada par de nodos. La representación esperada que tendremos es de forma matricial $M_{|V|\times|V|}$, tal que $M_{ij}$ almacenará la longitud del camino más corto desde el nodo $i$ al nodo $j$.

Una idea simple podría ser ejecutar $|V|$ veces un algoritmo para Single-source shortest path, uno desde cada nodo, y así obtener las distancias. Sin embargo, el método anterior tendría una complejidad de $O(|V||E|\log{|V|})$ para grafos no densos y $O(|V|^{3})$ para grafos densos **con aristas no negativas** (usando Dijkstra), mientras que tendría una complejidad de $O(|V|^{2}|E|)$ para grafos en general (usando Bellman-Ford).

Para este tipo de algoritmos, a diferencia de los SSSP, no se usará la lista de adyacencia de los grafos, sino su matriz de adyacencia. Para un grafo ponderado **sin bucles**, la matriz de adyacencia se define de la siguiente forma:

$$ M_{ij} = \left\{ \begin{array}{cc} 0 &\text{Si }i = j \\ w(i, j) &\text{Si }(i, j)\in E \\ \infty &\text{En caso contrario} \end{array} \right. $$

Además, tendremos una matriz de predecesor $\Pi$, definida de la siguiente forma:

$$ \Pi_{ij} = \left\{ \begin{array}{cc} NULL &\text{Si }i = j \\ \pi_{ij} &\text{Si }\pi_{ij}\text{ es predecesor de }j\text{ en algun camino a }i \end{array} \right. $$

Justo como la propiedad del grafo de predecesor, el subgrafo inducido por la $i$-ésima fila de $\Pi$ es un shortest-paths tree con raiz en $i$. Para cada nodo $i \in V$, definimos el **subgrafo de predecesor** de $G$ para $i$ como $G_{\pi, i} = (V_{\pi, i}, E_{\pi, i})$, donde:

$$ V_{\pi, i} = \{j \in V : \pi_{ij} \not = NULL\}\cup\{i\} $$

$$ E_{\pi, i} = \{(\pi_{ij}, j) : j \in V_{\pi, i}\backslash\{i\}\} $$

## Caminos más cortos y multiplicación de matrices

Vamos a analizar la estructura de los caminos más cortos para plantear una solución que use DP.

En SSSP probamos que todo subcamino de un camino más corto es también un camino más corto; si consideramos la cantidad de aristas que contiene dicho camino más corto, sea que $p$ tenga a lo mucho $m$ aristas, entonces podemos descomponer el camino $p$, que va de $i$ a $j$, en $p = p'_{ik} \leadsto k \rightarrow j$, donde

$$ w(p) = w(p'_{ik}) + w(k, j) $$

Lo que debemos notar en estos momentos es que el camino $p'_{ik}$ tiene a lo mucho $m-1$ aristas, por lo que podemos intentar analizar una solución con programación dinámica que contemple esta propiedad.

### Una solución recursiva

Sea $l_{ij}^{(m)}$ el peso del camino más corto del nodo $i$ al nodo $j$ de todos los caminos que tengan no más de $m$ aristas, para $m = 0$ tenemos caminos sin aristas, por lo tanto:

$$ l_{ij}^{(0)} = \left\{ \begin{array}{cc} 0 &i = j \\ \infty &i \not = j \end{array} \right. $$

Para $m \geq 1$, podemos considerar 2 casos para $l_{ij}^{(m)}$:

1) $l_{ij}^{(m)}$ tiene menos de $m$ aristas, entonces la respuesta tiene a lo mucho $(m - 1)$ aristas y es igual a $l_{ij}^{(m-1)}$.

2) $l_{ij}^{(m)}$ tiene exactamente $m$ aristas, entonces la respuesta se obtiene tomando un camino de longitud menor que $m$ y agregándole una arista del último nodo $k$ al nodo $j$. Dado que queremos la mínima respuesta posible, esta será igual a $\min\limits_{k \not = j, k \in V}{\{l_{ik}^{(m-1)} + w(k, j)\}}$.

Sin embargo, estos dos casos se reducen a la expresión del segundo pero para todos los posibles nodos, ya que $w(j,j) = 0$ y por lo tanto $l_{ij}^{(m-1)} + w(j, j) = l_{ij}^{(m-1)}$:

$$ l_{ij}^{(m)} = \min\limits_{k \in V}{\{l_{ik}^{(m-1)} + w(k, j)\}} $$

Ahora, debemos recordar de las propiedades demostradas en SSSP que si el grafo no tiene ciclos negativos, entonces para todo par de nodos $i$ y $j$ se cumplirá que $\delta(i, j) < \infty$ y además dicho camino más corto no tendrá más de $(n - 1)$ aristas.

### Calculando los caminos más cortos de forma iterativa

Considerando que nuestra entrada es la matriz de adyacencia $M$, procesaremos los valores de $L^{(1)}$, $L^{(2)}$, ..., $L^{(n-1)}$, donde $L^{(m)} = (l_{ij}^{(m)})$. Notemos que $L^(1) = M$, lo cual es fácil de verificar viendo los casos posibles (1 arista o ninguna arista).

La parte fundamental del algoritmo es el siguiente procedimiento, que recibe como argumentos a $L^{(m-1)}$ y $M$ para devolver $L^{(m)}$:

```Python
Extend-SP(L, M):
    n = L.rows
    Lp = matriz de n x n
    for i = 1 to n:
        for j = 1 to n:
            Lp[i][j] = inf
            for k = 1 to n:
                Lp[i][j] = min(Lp[i][j], L[i][k] + M[k][j])
    return Lp
```

Este procedimiento tiene una complejidad de $O(n^{3})$ por los 3 `for` anidados.

Notemos la similitud del algoritmo con la multiplicación de matrices cuadradas, la cual se puede generalizar de la siguiente forma:

```Python
Multiplication(A, B):
    n = A.rows
    C = matriz de n x n
    for i = 1 to n:
        for j = 1 to n:
            C[i][j] = inf
            for k = 1 to n:
                C[i][j] = f(C[i][j], A[i][k], B[k][j])
    return C
```

Donde $f$ debe ser una función asociativa y $inf$ una variable de inicialización que depende de $f$ (**no siempre es el elemento neutro de la operacion**)

De esta manera podemos definir nuestra propia operación $\cdot$ sobre nuestras matrices, tomando como referencia el algoritmo `Extend-SP`. No es difícil notar que para esta nueva operación se cumple que:

$$ L^{(1)} = L^{(0)} \cdot M = M $$
$$ L^{(2)} = L^{(1)} \cdot M = M \cdot M = M^{2} $$
$$ \vdots $$
$$ L^{(n-1)} = L^{(n-2)} \cdot M = M^{(n-1)} $$

Con lo anterior, podemos plantear un algoritmo que ejecute $(n - 2)$ veces el procedimiento `Extend-SP` para poder hallar $L^{(n-1)}$:

```Python
Slow-APSP(W):
    n = W.rows
    L = W
    for m = 2 to (n-1):
        L = Extend-SP(L, W)
    return L
```

Ya que no es necesario, no guardaremos todos los $L^{(m)}$, solamente reemplazaremos el $L^{(m-1)}$ actual con el nuevo $L^{(m)}$. Este algoritmo tiene una complejidad de $O(n^{4})$, lo cual no es tan eficiente como quisiéramos.

Podemos mejorar la complejidad usando **exponenciación rápida**, para reducirla a $O(n^{3}\log{n})$:

```Python
Fast-APSP(W):
    n = W.rows
    L = W
    m = 1
    while m < n - 1:
        L = Extend-SP(L, L)
        m = 2 * m
    return L
```

Este algoritmo justifica su correctitud por la definición de $L^{(m)}$, ya que para $m \geq n-1$ se dará que $L^{(n-1)} = L^{(m)}$.

# Algoritmo de Floyd-Warshall

El algoritmo de Floyd-Warshall considera que todos los caminos más cortos $p = \{v_{1}, v_{2}, \ldots, v_{l}\}$ con más de 1 arista deben tener al menos un **nodo intermedio**, es decir, un nodo en el conjunto $\{v_{2},\ldots,v_{l-1}\}$. La observación anterior ayuda a plantear lo siguiente:

Sea el conjunto de nodos $V = \{1,2,\ldots,n\}$, entonces analizaremos el subconjunto de vértices $\{1,2,\ldots, k\}$ para algún $k \geq 0$: para todo par de vértices $i$ y $j$, consideremos todos los caminos desde $i$ hasta $j$ tales que sus nodos intermedios pertenecen al conjunto $\{1,2,\ldots,k\}$ y sea $p$ un camino más corto entre todos ellos (evidentemente, $p$ es un camino simple). El algoritmo de Floyd-Warshall aprovecha la relación entre el camino $p$ y los caminos más cortos desde $i$ hasta $k$ con todos los nodos intermedios en el conjunto $\{1,2,\ldots,k-1\}$. La relación depende de si $k$ es o no un nodo intermedio del camino $p$.

* Si $k$ no es un vértice intermedio de $p$, entonces todos los vértices intermedios de $p$ están en el conjunto $\{1,2,\ldots,k-1\}$, por lo que un camino más corto con nodos intermedios en el conjunto $\{1,2,\ldots,k-1\}$ también es un camino más corto en el conjunto $\{1,2,\ldots,k\}$.

* Si $k$ es un vértice intermedio de $p$, entonces podemos particionar el camino en $i \xrightarrow{p_{1}} k \xrightarrow{p_{2}}j$. Notemos que $k$ no es vértice intermedio ni para $p_{1}$ ni para $p_{2}$, por lo que ambos son caminos más cortos en el conjunto $\{1,2,\ldots,k-1\}$

Lo anterior nos permite notar que todo camino más corto en el conjunto $\{1,2,\ldots,k\}$ es un solo camino más corto en el conjunto $\{1,2,\ldots,k-1\}$ o es la unión de dos caminos más cortos en $\{1,2,\ldots,k-1\}$ tomando al nodo $k$ como pivote.

## Una solución recursiva según Floyd-Warshall

Definamos a $d_{ij}^{(k)}$ como el peso del camino más corto de $i$ a $j$ en el conjunto $\{1,2,\ldots,k\}$; entonces, podemos notar que para $k = 0$ no hay nodos intermedios en los caminos, así que estos serán una sola arista o no existirán, lo cual nos señala que $d_{ij}^{(0)} = M_{ij}$. Para $k \geq 1$, podemos usar el segundo caso analizado en la sección anterior para obtener la siguiente recursión:

$$ d_{ij}^{(k)} = \left\{ \begin{array}{cc} M_{ij} & k = 0 \\ \min{\left\{d_{ij}^{(k-1)}, d_{ik}^{(k-1)} + d_{kj}^{(k-1)}\right\}} &k \geq 1 \end{array} \right. $$

## Implementación iterativa

Si bien hemos definido una solución recursiva, no es muy complicado pasar la idea a una implementación iterativa, ya que basta fijar el valor de $k$ y luego actualizar todos los pares $(i, j)$ en la nueva matriz:

```Python
Floyd-Warshall(W):
    n = W.rows
    D(0) = W
    for k = 1 to n:
        Sea D(k) una matriz de n x n
        for i = 1 to n:
            for j = 1 to n:
                D(k)[i][j] = min(D(k-1)[i][j], D(k-1)[i][k] + D(k-1)[k][j])
    return D(n)
```

Si consideramos la declaración de las matrices como un procedimiento proporcional al producto de sus dimensiones, el algoritmo tiene una complejidad total de $O(n^{3})$.

Una forma de optimizar la complejidad espacial que requiere el algoritmo es reciclar la matriz $D$, ya que al ser procesada para un valor $k$ fijado, este ya tendrá en la entrada $D_{ij}$ el resultado $d_{ij}^{(k-1)}$, por lo que bastará con actualizarlo con el valor $D_{ik} + D_{kj} = d_{ik}^{(k-1)} + d_{kj}^{(k-1)}$ y minimizar la respuesta final.

```Python
Floyd-Warshall(W):
    n = W.rows
    D = W
    for k = 1 to n:
        for i = 1 to n:
            for j = 1 to n:
                D[i][j] = min(D[i][j], D[i][k] + D[k][j])
    return D
```

# Algoritmo de Johnson para grafos poco densos

El algoritmo de Johnson plantea una alternativa al algoritmo de Floyd-Warshall cuando el grafo no es denso, es decir, no tiene tantas aristas. Usa como subrutinas los algoritmos de Dijkstra y de Bellman-Ford, bajo la premisa de que si se pueden **reetiquetar** los pesos de las aristas de forma que ninguna arista tenga peso negativo, entonces podríamos aplicar $|V|$ veces el algoritmo de Dijkstra para hallar todos los caminos más cortos entre cada par. Estos pesos nuevos, denotados por $\hat{w}$, deben cumplir lo siguiente:

1. Para todos los pares de vértices $u, v \in V$, el camino $p$ es un camino más corto de $u$ a $v$ usando los pesos $w$ si y solo si también lo es usando los pesos $\hat{w}$.

2. Para todas las aristas $(u, v) \in E$, $\hat{w}(u, v)$ es no negativo.

## Conservando los caminos más cortos reetiquetando los pesos

A partir de ahora denotaremos los caminos más cortos usando los pesos $w$ como $\delta$, mientras que aquellos usando los pesos $\hat{w}$ los denotaremos como $\hat{\delta}$. 

**Lema:** Dado un grafo dirigido y ponderado $G = (V, E)$ con función de pesos $w : E \to \mathbb{R}$ y sea $h : V \to \mathbb{R}$ cualquier función. Para cada arista $(u, v) \in E$, definamos:

$$ \hat{w}(u,v) = w(u, v) + h(u) - h(v) $$

Sea $p = \{v_{0}, v_{1}, \ldots, v_{k}\}$ un camino cualquiera de $v_{0}$ a $v_{k}$, entonces $p$ es un camino más corto usando los pesos $w$ si y solo si también lo es usando los pesos $\hat{w}$. Además, $G$ posee un ciclo negativo usando los pesos $w$ si y solo si también tiene uno usando los pesos $\hat{w}$.

**Prueba:**

Primero probaremos que:

$$ \hat{w}(p) = w(p) + h(v_{0}) - h(v_{k}) $$

Esto es algo simple, ya que:

$$ \hat{w}(p) = \sum\limits_{i = 1}^{k} \hat{w}(v_{i-1},v_{i}) $$

$$ \hat{w}(p) = \sum\limits_{i = 1}^{k} w(v_{i-1},v_{i}) + h(v_{i-1}) - h(v_{i}) $$

$$ \hat{w}(p) = \sum\limits_{i = 1}^{k} w(v_{i-1}, v_{i}) + h(v_{0}) - h(v_{k}) $$

$$ \hat{w}(p) = w(p) + h(v_{0}) - h(v_{k}) $$

Ya que para $v_{0}$ y $v_{k}$ fijos, el resultado $\hat{w}(p)$ solo varía si también lo hace $w(p)$, entonces si $p$ es camino más corto en $w$, también lo es en $\hat{w}(p)$, así que

$$ w(p) = \delta(v_{0}, v_{k}) \leftrightarrow \hat{w}(p) = \hat{\delta}(v_{0}, v_{k}) $$

Finalmente, si $c \in G$ es un ciclo de peso negativo usando los pesos $w$, tendremos que:

$$ \hat{w}(c) = w(c) + h(v_{0}) - h(v_{k}) $$

Pero al ser un ciclo, tenemos que $v_{0} = v_{k}$, por lo que:

$$ \hat{w}(c) = w(c) + h(v_{0}) - h(v_{k}) = w(c) $$

Así que el ciclo $c$ también tiene peso negativo usando los pesos $\hat{w}$.

## Generando pesos no negativos

Para generar los nuevos pesos $\hat{w}$ basta con definir una función $h : V \to \mathbb{R}$ adecuada, podemos lograr esto usando un método similar al del sistema de restricción de diferencias: Creamos un grafo nuevo $G' = (V', E')$ tal que:

$$ V' = V \cup \{s\} $$
$$ E' = E \cup \{(s, v) : v \in V\} $$

Además definimos los pesos de las nuevas aristas $w(s,v) = 0$.

Asumiento que $G$ no tiene ciclos negativos (en tal caso, $G'$ tampoco los tendrá, ya que solo puede tener ciclos que tomen nodos de $V$, pues el grado de entrada de $s$ es 0), podemos definir una función $h$ fácilmente:

Si asignamos $h(v) = \delta(s, v)$, por la desigualdad triangular, tenemos que, para toda arista $(u, v)$:

$$ \delta(s, v) \leq \delta(s, u) + w(u, v) \rightarrow 0 \leq w(u, v) + \delta(s, u) - \delta(s, v) = w(u, v) + h(u) - h(v) = \hat{w}(u, v) $$

Así que esto nos permite generar pesos no negativos.

## Calculando los caminos más cortos

Dado que ahora nuestras aristas tienen pesos no negativos, podemos aplicar el algoritmo de Dijkstra desde cada nodo para rellenar la tabla de caminos más cortos $D_{ij}$. El algoritmo tomará $O(VE\log{V} + VE)$ en total, aunque puede ser reducido a $O(V^{2}\log{V} + VE)$ usando Fibonacci Heap.

```Python
Johnson():
    for v in V:
        d[v] = 0
    if Bellman() == False:
        return Error
    else:
        for v in V:
            h[v] = d[v]
        for (u, v) in E:
            w2(u,v) = w(u, v) + h[u] - h[v]
        for u in V:
            Dijkstra(u, w2) # Este Dijkstra usa los pesos almacenados en su argumento
            for v in V:
                D[u][v] = d[v] + h[v] - h[u] # Notemos que sumamos -(h[u] - h[v]) segun la reetiqueta
```

Notemos que en el algoritmo mostrado arriba se usa el arreglo $d$ como una variable que mantendrá las distancias más cortas procesadas en diferentes fases: Primero las obtenidas con el algoritmo de Bellman-Ford y luego las procesadas por cada ejecución del algoritmo de Dijkstra.

## Ejercicios para practicar:

- [Contest ADA University - Floyd Warshall](https://www.e-olymp.com/en/contests/12828)